In [39]:
import torch
torch.cuda.empty_cache()
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import random_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform_Training = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


transform_Testing = transforms.Compose([
    transforms.ToTensor(),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_Training)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_Testing) 

Files already downloaded and verified
Files already downloaded and verified


In [40]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=5, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=5, shuffle=False)

In [47]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def _make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v

class InvertedResidual(nn.Module):
    def __init__(self, in_channels, out_channels, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        hidden_dim = round(in_channels * expand_ratio)
        self.use_res_connect = stride == 1 and in_channels == out_channels

        layers = []
        if expand_ratio != 1:
            # pw
            layers.append(nn.Conv2d(in_channels, hidden_dim, kernel_size=1, stride=1, padding=0, bias=False))
            layers.append(nn.BatchNorm2d(hidden_dim))
            layers.append(nn.ReLU6(inplace=True))
        layers.extend([
            # dw
            nn.Conv2d(hidden_dim, hidden_dim, kernel_size=3, stride=stride, padding=1, groups=hidden_dim, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True),
            # pw-linear
            nn.Conv2d(hidden_dim, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels),
        ])
        self.conv = nn.Sequential(*layers)

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)

class MobileNetV2(nn.Module):
    def __init__(self, num_classes=10):
        super(MobileNetV2, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU6(inplace=True),
            InvertedResidual(32, 16, 1, 1),
            InvertedResidual(16, 24, 1, 6),
            InvertedResidual(24, 24, 1, 6),
            InvertedResidual(24, 32, 2, 6),
            InvertedResidual(32, 32, 1, 6),
            InvertedResidual(32, 32, 1, 6),
            InvertedResidual(32, 64, 2, 6),
            InvertedResidual(64, 64, 1, 6),
            InvertedResidual(64, 64, 1, 6),
            InvertedResidual(64, 64, 1, 6),
            InvertedResidual(64, 96, 1, 6),
            InvertedResidual(96, 96, 1, 6),
            InvertedResidual(96, 96, 1, 6),
            InvertedResidual(96, 160, 2, 6),
            InvertedResidual(160, 160, 1, 6),
            InvertedResidual(160, 160, 1, 6),
            InvertedResidual(160, 320, 1, 6),
            nn.Conv2d(320, 1280, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(1280),
            nn.ReLU6(inplace=True),
        )
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
      
        self.classifier = nn.Linear(1280, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
      
        x = self.classifier(x)
        return x





In [49]:
# Assuming the device setup as before
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Model instantiation and moving to the appropriate device
model = MobileNetV2().to(device)

In [51]:
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.045, alpha=0.9, weight_decay=0.00004, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)


In [34]:
def train(model, device, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        correct = 0
        total = 0
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
              .format(epoch + 1, num_epochs, total_loss / (batch_idx + 1), 100. * correct / total))

In [36]:
train(model, device, trainloader, criterion, optimizer, num_epochs=15)

Epoch [1/15], Loss: 0.6592, Accuracy: 77.34%
Epoch [2/15], Loss: 0.6000, Accuracy: 79.41%
Epoch [3/15], Loss: 0.5557, Accuracy: 80.92%
Epoch [4/15], Loss: 0.5107, Accuracy: 82.33%
Epoch [5/15], Loss: 0.4773, Accuracy: 83.30%
Epoch [6/15], Loss: 0.4435, Accuracy: 84.56%
Epoch [7/15], Loss: 0.4141, Accuracy: 85.62%
Epoch [8/15], Loss: 0.3808, Accuracy: 86.85%
Epoch [9/15], Loss: 0.3595, Accuracy: 87.50%
Epoch [10/15], Loss: 0.3360, Accuracy: 88.27%
Epoch [11/15], Loss: 0.3152, Accuracy: 88.84%
Epoch [12/15], Loss: 0.2939, Accuracy: 89.62%
Epoch [13/15], Loss: 0.2776, Accuracy: 90.21%
Epoch [14/15], Loss: 0.2604, Accuracy: 90.89%
Epoch [15/15], Loss: 0.2468, Accuracy: 91.30%


In [37]:
# Continue training for an additional 5 epochs
train(model, device, trainloader, criterion, optimizer, num_epochs=5)


Epoch [1/5], Loss: 0.2300, Accuracy: 91.66%
Epoch [2/5], Loss: 0.2214, Accuracy: 92.05%
Epoch [3/5], Loss: 0.2113, Accuracy: 92.51%
Epoch [4/5], Loss: 0.2013, Accuracy: 92.80%
Epoch [5/5], Loss: 0.1903, Accuracy: 93.28%


In [38]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():  # No need to track gradients for validation
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy of the model on the test images: {test_accuracy:.2f}%')


Test Accuracy of the model on the test images: 66.45%
